## Лаба 1. Расчет рейтингов фильмов – RDD

In [56]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [57]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("Laba_1")
         .getOrCreate())

In [58]:
sc = spark.sparkContext

In [59]:
spark

In [60]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [61]:
rdd = sc.textFile("/labs/laba01/ml-100k/u.data").map(lambda x: x.split('\t'))

In [62]:
rdd.take(10)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596'],
 ['298', '474', '4', '884182806'],
 ['115', '265', '2', '881171488'],
 ['253', '465', '5', '891628467'],
 ['305', '451', '3', '886324817'],
 ['6', '86', '3', '883603013']]

In [63]:
rdd.count()

100000

In [65]:
film_id = '318'

In [66]:
rdd_1 = rdd.filter(lambda x: x[1] == film_id)\
            .map(lambda x: (x[2], 1))\
            .reduceByKey(lambda x, y: x + y)\
            .collect()

In [67]:
rdd_1.sort()

In [68]:
rdd_1

[('1', 4), ('2', 6), ('3', 23), ('4', 79), ('5', 186)]

In [69]:
rdd_2 = rdd.map(lambda x: (x[2], 1))\
            .reduceByKey(lambda x, y: x + y)\
            .collect()

In [70]:
rdd_2.sort()

In [71]:
rdd_2

[('1', 6110), ('2', 11370), ('3', 27145), ('4', 34174), ('5', 21201)]

In [72]:
result = {
   "hist_film": [x[1] for x in rdd_1],
   "hist_all": [x[1] for x in rdd_2]
}

In [73]:
result

{'hist_film': [4, 6, 23, 79, 186],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [45]:
import json
with open('lab01.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(result))

In [74]:
spark.stop()